<div style="text-align: center;margin:10px 0px 0px; padding-top:10px 0px 0px">
<b style="font-size: 56px; font-weight: bold; margin-bottom: 0px; padding-bottom: 0px">Trabajo Final</b><br>
<b style="font-size: 18px; font-weight: bold; margin-top: 0px; padding-top: 0px">Visualización de datos</b>
</div>

<hr style="border-top: 2px solid #333; margin-top: 0px; margin-bottom: 0px">

# Introducción


Este trabajo tiene como objetivo desarrollar un dashboard interactivo utilizando Plotly Dash para visualizar los KPIs de ventas de una empresa global de alimentación. La estructura de importación y configuración está diseñada para cumplir con los requisitos básicos y optimizar la organización del código.
        

# 1. Importar librerías

In [172]:
# Importamos dash
import dash
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
from dash import jupyter_dash
from dash import dash_table

# Importamos plotly
import plotly.express as px
import plotly.graph_objects as go

# Importamos librerias para la manipulación de datos
import pandas as pd
import numpy as np

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

# 2. Carga de datos

In [173]:
# Cargamos ambos datases
df_1= pd.read_csv('../data/sales_1.csv',sep=",")
df_2= pd.read_csv('../data/sales_2.csv',sep=",")
# Fusionamos los datasets en uno único
df= pd.concat([df_1,df_2])
# Establecemos la columna de date a datetime ya que será más cómodo de trabajar con fechas
df['date'] = pd.to_datetime(df['date'])


# 3. Dashboard Set up

In [174]:
# Configuración básica de la aplicación
app = dash.Dash(__name__,suppress_callback_exceptions=True)

# Título de la Ventana
app.title = "Sales Analytics Dashboard"

# Estilo del contenedor de pestañas
tabs_styles = {
    'height': '60px', 
    'backgroundColor': 'transparent',
    'borderRadius': '8px', 
    'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)',  # Sombra
    'padding': '10px', 
}

# Estilo de pestañas no seleccionadas
tab_style = {
    'display': 'flex',  # Alinear elementos
    'alignItems': 'center',  # Centrado vertical
    'justifyContent': 'center',  # Centrado horizontal
    'borderBottom': '2px solid #e0e0e0',  # Línea gris inferior
    'padding': '15px 25px',  # Márgenes ampliados
    'fontWeight': 'bold',
    'fontSize': '18px',  
    'color': '#4b565e', 
    'backgroundColor': '#ffffff',  
    'borderRadius': '8px',  
    'margin': '7px',  # Separación entre pestañas
    'transition': 'all 0.3s ease',  # Transición suave
    'cursor': 'pointer',  # Indicador de clic
}

# Estilo de pestañas seleccionadas
tab_selected_style = {
    'display': 'flex',  
    'alignItems': 'center',  
    'justifyContent': 'center',  
    'borderTop': '4px solid #00845b',  
    'borderBottom': '3px solid #00845b', 
    'backgroundColor': '#eaf5f0', 
    'color': '#00845b', 
    'padding': '15px 25px',  
    'fontSize': '18px',  
    'fontWeight': 'bold',
    'boxShadow': '0 6px 14px rgba(0, 0, 0, 0.2)',  
    'borderRadius': '8px', 
    'margin': '7px', 
    'transition': 'all 0.3s ease',
    'cursor': 'pointer',
}

# Layout de la aplicación
app.layout = html.Div([
    # Título principal del dashboard
    html.Header(
        html.H1('Sales Analytics Dashboard', style={
            'textAlign': 'center',
            'color': '#ffffff',
            'padding': '5px 0',
            'font-family': 'Georgia, serif',
            'font-weight': 'bold',
            'fontSize': '42px',
            'textShadow': '3px 3px 7px rgba(0, 0, 0, 0.4)',
        }),
        style={
            'backgroundColor': '#006d5b',
            'boxShadow': '0 4px 12px rgba(0, 0, 0, 0.4)',
            'borderRadius': '15px',
            'margin': '20px',
            'overflow': 'hidden'
        }
    ),

    # Pestañas
    dcc.Tabs(id="tabs-example-graph", value='tab_1', children=[
        dcc.Tab(label='Resumen de Métricas', value='tab_1', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Análisis por Tienda', value='tab_2', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Análisis Avanzado', value='tab_3', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Pestaña Explicativa', value='tab_4', style=tab_style, selected_style=tab_selected_style)
    ], style=tabs_styles),
    
    # Contenido de las pestañas
    html.Div(id='tabs-content-example-graph')
])

# Layout
layout = go.Layout(
    margin=go.layout.Margin(
        l=60,  
        r=60,  
        b=60,  
        t=60  
    ),
    title_font=dict(size=20, color='#006d5b', family="Arial"),
    font=dict(color='#4b565e', size=12),
    plot_bgcolor='#eaf4eb',  
    paper_bgcolor='#eaf4eb',
)

# 4. Funciones genéricas

Con el objetivo de conseguir la eficiencia máxima y, con ello, evitar la repetición de código, definimos aquellas funciones que se vayan a repetir a lo largo del código.

In [175]:
def firma_pag():
    """
    Firma a pie de página con separador horizontal, nombre y año.
    """
    return html.Div([
        html.Hr(style={'borderTop': '1px solid #006d5b', 'marginTop': '30px', 'marginBottom': '10px'}),
        html.Div([
            html.Span('Íñigo de Oñate', style={'color': '#4b565e', 'fontSize': '16px'}),
            html.Span('2024', style={'color': '#4b565e', 'fontSize': '16px', 'float': 'right'})
        ], style={'display': 'flex', 'justify-content': 'space-between', 'alignItems': 'center'})
    ])

def title(tab_name):
    """
    Creación del título principal de las pestañas
    """
    return html.Div([
        html.H1(tab_name, 
            style={
                'text-align': 'center', 
                'color': '#4b565e', 
                'fontSize': '42px',
                'marginBottom': '20px'
            }
        ),
        html.Hr(style={'borderTop': '4px solid #006d5b', 'marginTop': '30px', 'marginBottom': '30px'})
    ])

def create_dropdown_with_label(label_text,label_size, dropdown_id, options, default_value, container_style=None, dropdown_style=None):
    """
    Creación de texto y dropdown.
    
    :param label_text: Texto del encabezado.
    :param label_size: Tamaño del texto.
    :param dropdown_id: ID único para el dropdown.
    :param options: Lista de opciones para el dropdown (formato [{'label': 'Texto', 'value': 'valor'}]).
    :param default_value: Valor seleccionado por defecto en el dropdown.
    :param container_style: Estilo adicional para el contenedor principal.
    :param dropdown_style: Estilo adicional para el dropdown.
    :return: Componente Dash.
    """
    return html.Div([
        html.Div([
            html.H3(label_text, style={'color': '#006d5b', 'fontSize': label_size, 'marginRight': '20px'}),
            dcc.Dropdown(
                id=dropdown_id, # Identificador único 
                options=options,
                value=default_value,
                style=dropdown_style or {'width': '300px'} 
            )
        ], style=container_style or {'display': 'flex', 'alignItems': 'center', 'justifyContent': 'flex-start', 'marginBottom': '20px'})
    ])

def create_section(title, paragraphs):
    """
    Creación de bloques de texto explicativo
    """
    return html.Div([
        html.H3(title, style={'color': '#006d5b'}),
        *[html.P(paragraph) for paragraph in paragraphs]
    ], style={'marginBottom': '20px'}) 


# 5. Pestañas

Dividiremos el código por cada pestaña, de tal forma que definimos las variables de contenido *tab_nºpestaña_content* para las respectivas.

# Primera Pestaña: Resumen de Métricas

## Funciones

In [176]:
# Contador de métricas básicas
def contador():
    # Calculamos el número total de tiendas, productos y estados
    num_tiendas = df["store_nbr"].nunique()
    num_productos = df["family"].nunique()
    num_estados = df["state"].nunique()

    return num_tiendas,num_productos,num_estados

# Gráfico de los 10 productos más vendidos
def ranking_productos():
    # Calculamos el promedio de ventas por producto y seleccionamos los 10 más vendidos
    ranking_ventas = df.groupby("family")["sales"].mean().sort_values(ascending=False).reset_index().head(10)
    
    # Creamos un gráfico de barras para mostrar el ranking
    barChart = dcc.Graph(
        figure=go.Figure(layout=layout).add_trace(
            go.Bar(
                x=ranking_ventas["family"],  # Productos
                y=ranking_ventas["sales"],  # Ventas promedio
                marker=dict(color='#006d5b')  
            )
        ).update_layout(
            xaxis_title="Productos",  # Etiqueta del eje X
            yaxis_title="Ventas Medias",  # Etiqueta del eje Y
            xaxis=dict(showgrid=False),  # Eliminamos cuadrículas
            yaxis=dict(showgrid=False),  
        ),
        style={'width': '48%', 'height': '40vh', 'display': 'inline-block'}  # Estilo del gráfico
    )
    return barChart

# Gráfico de líneas de ventas mensuales
def ventas_mensuales():
    # Agrupamos las ventas totales por mes
    ventas_mes = df.groupby("month")["sales"].sum().reset_index()
    
    # Convertimos las ventas a millones (mejor visualización)
    ventas_mes["sales"] = ventas_mes["sales"] / 1e6

    # Gráfico de líneas con marcadores para representar las ventas por mes
    lineChart = dcc.Graph(
        figure=go.Figure(layout=layout).add_trace(
            go.Scatter(
                x=ventas_mes["month"],  # Meses
                y=ventas_mes["sales"],  # Ventas totales
                mode="lines+markers",  
                marker=dict(color='#006d5b', size=8), 
                line=dict(width=2) 
            )
        ).update_layout(
            title="Ventas Totales por Mes",  
            xaxis_title="Mes", 
            yaxis_title="Ventas Totales (Millones)", 
            xaxis=dict(
                tickmode="array",  # Visualización de ticks
                tickvals=list(range(1, 13)),  
                ticktext=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'],  # Etiquetas de los meses
                showgrid=False  
            ),
            yaxis=dict(
                showgrid=False, 
                ticksuffix="M"  # Sufijo "M" de millones
            )
        ),
        style={'width': '48%', 'height': '40vh', 'display': 'inline-block'}  
    )
    return lineChart

# Gráfico de ventas por día de la semana
def ventas_por_dia_semana():
    # Calculamos las ventas totales por día de la semana
    ventas_dia_semana = df.groupby("day_of_week")["sales"].sum().reset_index()
    
    # Creamos un gráfico de líneas para representar las ventas por día
    lineChart = dcc.Graph(
        figure=go.Figure(layout=layout).add_trace(
            go.Scatter(
                x=ventas_dia_semana["day_of_week"],  # Días de la semana en el eje X
                y=ventas_dia_semana["sales"],  # Ventas totales en el eje Y
                mode="lines+markers",  
                marker=dict(color='#006d5b')  
            )
        ).update_layout(
            xaxis_title="Día de la Semana",  
            yaxis_title="Ventas Totales", 
            xaxis=dict(showgrid=False), 
            yaxis=dict(showgrid=False), 
        ),
        style={'width': '48%', 'height': '40vh', 'display': 'inline-block'}  
    )
    return lineChart

# Gráfico de ventas por estado
def ventas_por_estado():
    # Calculamos las ventas totales por estado y las ordenamos en orden descendente
    ventas_estado = df.groupby("state")["sales"].sum().sort_values(ascending=False).reset_index()
    
    # Creamos un gráfico de barras para representar las ventas por estado
    barChart = dcc.Graph(
        figure=go.Figure(layout=layout).add_trace(
            go.Bar(
                x=ventas_estado["state"],  # Estados en el eje X
                y=ventas_estado["sales"],  # Ventas totales en el eje Y
                marker=dict(color='#006d5b')  
            )
        ).update_layout(
            xaxis_title="Estado", 
            yaxis_title="Ventas Totales", 
            xaxis=dict(showgrid=False), 
            yaxis=dict(showgrid=False), 
        ),
        style={'width': '48%', 'height': '40vh', 'display': 'inline-block'}  
    )
    return barChart

# Gráfico de ventas durante días laborales vs festivos
def ventas_dias_festivos():
    # Agrupamos las ventas totales por tipo de día (laboral o festivo)
    df_festivos = df.groupby("holiday_type")["sales"].sum().reset_index()
    
    # Creamos un gráfico de barras para comparar ventas entre tipos de día
    barChart = dcc.Graph(
        figure=go.Figure(layout=layout).add_trace(
            go.Bar(
                x=df_festivos["holiday_type"],  # Tipos de día en el eje X
                y=df_festivos["sales"],  # Ventas totales en el eje Y
                marker=dict(color='#006d5b') 
            )
        ).update_layout(
            xaxis_title="Tipo de Día",  
            yaxis_title="Ventas Totales",
            xaxis=dict(showgrid=False),  
            yaxis=dict(showgrid=False),  
        ),
        style={'width': '48%', 'height': '40vh', 'display': 'inline-block'}  
    )
    return barChart


In [177]:
tab_1_content = html.Div([

    # Título principal de la pestaña
    title("Resumen de Métricas"),
    
    # Métricas clave: número de tiendas, productos y estados
    html.Div([
        html.Div([
            html.H3(
                ["Nº de tiendas", "Nº total de productos", "Total de Estados"][i], 
                style={'text-align': 'center', 'color': '#006d5b', 'fontSize': '28px'}
            ),
            html.H2(
                contador()[i], 
                style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '52px'}
            )
        ], style={'width': '30%', 'text-align': 'center'}) for i in range(3)
    ], style={'display': 'flex', 'justify-content': 'space-around', 'marginTop': '20px', 'marginBottom': '20px'}),
    
    # Línea horizontal decorativa
    html.Hr(style={'borderTop': '3px solid #006d5b', 'marginTop': '30px', 'marginBottom': '30px'}),
    
    # Gráficos organizados en filas
    html.Div([
        # Primera fila
        html.Div([
            html.Div([
                html.H3('10 Productos más vendidos', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
                dcc.Graph(
                    id='top_10_productos',
                    figure=ranking_productos().figure,
                    style={'width': '100%', 'height': '300px'}
                )
            ], style={'border': '1px solid #006d5b', 'padding': '10px', 'borderRadius': '10px', 
                      'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '45%', 'marginBottom': '20px'}),
            
            html.Div([
                html.H3('Ventas mensuales', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
                dcc.Graph(
                    id='ventas_mensuales',
                    figure=ventas_mensuales().figure,
                    style={'width': '100%', 'height': '300px'}
                )
            ], style={'border': '1px solid #006d5b', 'padding': '10px', 'borderRadius': '10px', 
                      'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '45%', 'marginBottom': '20px'})
        ], style={'display': 'flex', 'justify-content': 'space-around'}),
        
        # Segunda fila
        html.Div([
            html.Div([
                html.H3('Ventas por día de la semana', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
                dcc.Graph(
                    id='ventas_por_dia_semana',
                    figure=ventas_por_dia_semana().figure,
                    style={'width': '100%', 'height': '300px'}
                )
            ], style={'border': '1px solid #006d5b', 'padding': '10px', 'borderRadius': '10px', 
                      'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '45%', 'marginBottom': '20px'}),
            
            html.Div([
                html.H3('Ventas por estado', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
                dcc.Graph(
                    id='ventas_por_estado',
                    figure=ventas_por_estado().figure,
                    style={'width': '100%', 'height': '300px'}
                )
            ], style={'border': '1px solid #006d5b', 'padding': '10px', 'borderRadius': '10px', 
                      'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '45%', 'marginBottom': '20px'})
        ], style={'display': 'flex', 'justify-content': 'space-around'}),
        
        # Tercera fila
        html.Div([
            html.Div([
                html.H3('Ventas en días laborales y festivos', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
                dcc.Graph(
                    id='ventas_dias_festivos',
                    figure=ventas_dias_festivos().figure,
                    style={'width': '100%', 'height': '300px'}
                )
            ], style={'border': '1px solid #006d5b', 'padding': '10px', 'borderRadius': '10px', 
                      'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '95%', 'marginBottom': '20px'})
        ], style={'display': 'flex', 'justify-content': 'space-around'}),
    ]),
    
    # Firma a pie de página
    firma_pag(),
    
], style={'background-color': '#eaf4eb', 'padding': '20px', 'border-radius': '20px', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'margin-top': '10px'})


# Segunda Pestaña: Análisis por Tienda

In [178]:
tab_2_content = html.Div([

    # Título de la pestaña
    title("Análisis por Tienda"),

    # Dropdown para seleccionar una tienda específica
    create_dropdown_with_label(label_text="Seleccione una tienda para ver el análisis",label_size="30px",dropdown_id="dropdown-tienda",options=[{'label': i, 'value': i} for i in sorted(df['store_nbr'].unique())],default_value=1,container_style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'flex-start', 'padding': '10px 20px', 'marginBottom': '30px'}, dropdown_style={'width': '50%'}),
            
    # Gráfico de Ventas Anuales por Tienda
    html.Div([
        html.H3('Ventas anuales por tienda', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
        dcc.Graph(
            id='ventas-anuales',
            style={'width': '100%', 'height': '450px'}
        )
    ], style={'border': '1px solid #006d5b', 'padding': '20px', 'borderRadius': '10px', 
              'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '95%', 'margin': 'auto', 'marginBottom': '20px'}),

    # Gráfico de Ventas por Producto
    html.Div([
        html.H3('Ventas por producto', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
        dcc.Graph(
            id='ventas-productos',
            style={'width': '100%', 'height': '450px'}
        )
    ], style={'border': '1px solid #006d5b', 'padding': '20px', 'borderRadius': '10px', 
              'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '95%', 'margin': 'auto', 'marginBottom': '20px'}),

    # Gráfico de Productos en Promoción
    html.Div([
        html.H3('Productos en promoción', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
        dcc.Graph(
            id='productos-promocion',
            style={'width': '100%', 'height': '450px'}
        )
    ], style={'border': '1px solid #006d5b', 'padding': '20px', 'borderRadius': '10px', 
              'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '95%', 'margin': 'auto', 'marginBottom': '20px'}),

    # Tabla resumen de productos en promoción
    html.Div([
        html.H3('Resumen de productos en promoción', style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
        dash_table.DataTable(
            id='tabla-productos-promocion',
            columns=[
                {'name': 'Familia', 'id': 'family'}, # Categoría del producto
                {'name': 'Ventas (% Promoción)', 'id': 'sales'}, # Porcentaje de ventas en promoción
                {'name': 'Nº Promociones', 'id': 'onpromotion'} # Número total de promociones
            ],
            style_cell={'textAlign': 'center', 'padding': '10px', 'minWidth': '90px', 'width': '90px', 'maxWidth': '100px'},
            style_table={'overflowX': 'auto', 'width': '100%'}, # Habilitamos el scroll horizontal
            style_header={'backgroundColor': '#006d5b', 'color': 'white', 'fontWeight': 'bold'},
            style_data={'backgroundColor': '#f5f5f5'},
            page_size=10 # Nº filas visibles por página
        )
    ], style={'border': '1px solid #006d5b', 'padding': '20px', 'borderRadius': '10px',
            'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '95%', 'margin': 'auto', 'marginBottom': '20px'}),

    # Firma a pie de página
    firma_pag(),
    
], style={'background-color': '#eaf4eb', 'padding': '20px','border-radius': '20px', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)','margin-top': '10px'})

# Callback que actualiza los gráficos y la tabla al seleccionar una tienda del dropdown
@app.callback(
    [Output('ventas-anuales', 'figure'),
     Output('ventas-productos', 'figure'),
     Output('productos-promocion', 'figure'),
     Output('tabla-productos-promocion', 'data')],
    Input('dropdown-tienda', 'value')
)
def update_visualizations(tienda):
    # Filtramos los datos para la tienda seleccionada
    df_filtrado = df[df['store_nbr'] == tienda]

    # Usaremos una paleta de colores personalizada que van a juego con los colores de la presentación
    green_palette = [
        '#004d40', '#006d5b', '#00897b', '#26a69a', '#4caf50',
        '#66bb6a', '#81c784', '#a5d6a7', '#1b5e20', '#2e7d32', 
        '#388e3c', '#43a047', '#4caf50', '#66bb6a', '#76c68f', 
        '#98e097', '#baf2c1', '#dcffe8'
    ]

    # Ventas Anuales

    # Agrupamos las ventas por año y las convertimos a millones para simplificar la visualización
    ventas_year = df_filtrado.groupby("year")["sales"].sum().reset_index()
    ventas_year["sales"] = ventas_year["sales"] / 1e6

    #  Creamos el gráfico de barras de ventas anuales, usando la paleta de colores previamente declarada
    fig_ventas_anuales = px.bar(
        ventas_year,
        x="year",
        y="sales",
        labels={'year': 'Año', 'sales': 'Ventas (Millones)'},
        title=f'Ventas Anuales de la Tienda {tienda}', # Título dinámico basado en la tienda seleccionada
        color_discrete_sequence=green_palette
    )

    # Fondo transparente y color del texto
    fig_ventas_anuales.update_layout(
        plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font_color='#4b565e'
    )

    # Ventas por Producto

    # Agrupamos las ventas por familia de productos
    ventas_productos = df_filtrado.groupby("family")["sales"].sum().reset_index()
    ventas_productos["sales"] = ventas_productos["sales"] / 1e6  # Convertimos a millones

    # Gráfico de barras de ventas por familia de productos
    fig_ventas_productos = px.bar(
        ventas_productos,
        x="sales",
        y="family",
        orientation='h',
        color="family", # Usamos el color para distinguir las diferentes familias 
        labels={'sales': 'Ventas (Millones)', 'family': 'Familia de Productos'},
        title=f'Ventas por Producto en la Tienda {tienda}', # Título dinámico
        color_discrete_sequence=green_palette
    )

    # Igual que antes, definimos fondo transparente y color del texto
    fig_ventas_productos.update_layout(
        plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font_color='#4b565e'
    )

    # Productos en Promoción

    # Filtramos las ventas que tienen productos en promoción y agrupamos por familia
    productos_promocion = df_filtrado[df_filtrado["onpromotion"] != 0].groupby("family")["sales"].sum().reset_index()
    productos_promocion["sales"] = productos_promocion["sales"] / 1e6

    # Gráfico de barras de ventas de productos en promoción
    fig_productos_promocion = px.bar(
        productos_promocion,
        x="sales",
        y="family",
        orientation='h',
        color="family",
        labels={'sales': 'Ventas (Millones)', 'family': 'Familia de Productos'},
        title=f'Productos en Promoción en la Tienda {tienda}', # Título dinámico
        color_discrete_sequence=green_palette
    )

    # Configuramos el fondo del gráfico como transparente y ajustamos el color del texto
    fig_productos_promocion.update_layout(
        plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font_color='#4b565e'
    )

    # Tabla de Productos en Promoción

    # Creamos un resumen de productos en promoción con ventas totales y la cantidad de productos promocionados por familia
    resumen_promocion = df_filtrado[df_filtrado["onpromotion"] != 0].groupby("family").agg(
        sales=('sales', 'sum'),
        onpromotion=('onpromotion', 'sum') # Sumamos el número de promociones para cada familia
    ).reset_index()

    # Calculamos el porcentaje de ventas por cada familia respecto al total de ventas en promoción
    total_sales = resumen_promocion["sales"].sum()
    if total_sales > 0: # Evitamos divisiones por cero.
        resumen_promocion["sales"] = (resumen_promocion["sales"] / total_sales) * 100

    # Redondeamos las ventas al 2do decimal
    resumen_promocion["sales"] = resumen_promocion["sales"].round(2)

    # Convertimos la tabla a formato dict para el dataTable
    tabla_data = resumen_promocion.to_dict('records')

    return (
        fig_ventas_anuales,
        fig_ventas_productos,
        fig_productos_promocion,
        tabla_data
    )



# Tercera Pestaña: Análisis Avanzado

In [179]:
tab_3_content = html.Div([
    # Título principal de la pestaña
    title("Visualización por Estados"),

    # Dropdown para filtrar datos por estado
    create_dropdown_with_label(label_text="Seleccione un estado",label_size="30px",dropdown_id="dropdown",
                               options=[{'label': i, 'value': i} for i in sorted(df['state'].unique())],
                               default_value="Pichincha"),

    # Gráfico: Análisis de estacionalidad
    html.Div([
        html.H3("Análisis de estacionalidad", style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),

        # Checklist para elegir las variables a dibujar
        dcc.Checklist(
            id='estacionalidad-variables',
            options=[
                {'label': 'Ventas diarias', 'value': 'sales'},
                {'label': 'Productos en promoción', 'value': 'onpromotion'},
                {'label': 'Días festivos', 'value': 'holidays'}
            ],
            value=['sales'],  # Por defecto, mostramos las ventas diarias
            inline=True,  
            style={
                'text-align': 'center', 
                'marginBottom': '20px'
            },
            inputStyle={
                'marginRight': '10px',  # Espacio entre el cuadro de selección y la etiqueta
            },
            labelStyle={
                'display': 'inline-block', # Permitimos espaciado personalizado
                'marginRight': '15px',     # Espacio entre opciones
                'padding': '5px 10px',     # Espaciado interno para estilo del fondo
                'borderRadius': '5px',     # Bordes redondeados
                'cursor': 'pointer'        # Cambiar cursor al pasar sobre la opción
            },
            persistence=True,  # Mantenemos la selección en caso de recarga
            persistence_type='session'  # Persistencia por sesión
        ),

        # Actualiza los datos según los filtros
        dcc.Graph(
            id='grafico-estacionalidad',
            style={'width': '100%', 'height': '450px'}
        ),

        # Slider para seleccionar rango de fechas
        html.Div(
            dcc.RangeSlider(
                id='range-slider-estacionalidad',
                min=df['date'].min().timestamp(),
                max=df['date'].max().timestamp(),
                value=[df['date'].min().timestamp(), df['date'].max().timestamp()],
                marks={ # Intervalos de 6 meses para mayor claridad
                    int(date.timestamp()): date.strftime('%b-%Y')  # Formato abreviado del mes y año
                    for date in pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='6M')
                },
                step=None,  
                tooltip={"always_visible": True}  # Tooltip siempre visible para mostrar el rango seleccionado
            ),
            style={'marginTop': '20px', 'marginBottom': '20px'}  
        )
    ], style={'border': '1px solid #006d5b', 'padding': '20px', 'borderRadius': '10px', 
            'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '95%', 'margin': 'auto', 'marginBottom': '20px'}),

    # Gráfico: Análisis comparativo multidimensional
    html.Div([
        html.H3("Análisis comparativo multidimensional", style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
        create_dropdown_with_label(
            label_text="Agrupar por:",
            label_size="20px",
            dropdown_id="agrupacion-selector",
            options=[
                {'label': 'No agrupar', 'value': 'none'},  # Opción inicial (sin agrupar)
                {'label': 'Clúster', 'value': 'cluster'},
                {'label': 'Tipo de Tienda', 'value': 'store_type'}
            ],
            default_value='none',  # Por defecto
            container_style={'justifyContent': 'center'},
            dropdown_style={'width': '250px'}
        ),
        # Actualiza los datos según los filtros
        dcc.Graph(
            id='grafico-comparativo',
            style={'width': '100%', 'height': '450px'}
        ),
        # Slider para ajustar el rango temporal del análisis (misma lógica que antes)
        html.Div(
            dcc.RangeSlider(
                id='range-slider-comp',
                min=df['date'].min().timestamp(),
                max=df['date'].max().timestamp(),
                value=[df['date'].min().timestamp(), df['date'].max().timestamp()],
                marks={
                    int(date.timestamp()): date.strftime('%b %Y')  # Formato abreviado de mes y año
                    for date in pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='6M')
                },
                step=None,  
                tooltip={"always_visible": True}  
            ),
            style={'marginTop': '20px', 'marginBottom': '20px'}  # Espaciado
        )
    ], style={'border': '1px solid #006d5b', 'padding': '20px', 'borderRadius': '10px', 
              'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)', 'width': '95%', 'margin': 'auto', 'marginBottom': '20px'}),

    # Gráfico: Patrones de venta
    html.Div([
        html.H3("Patrones de venta", style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
        # Primera línea: Dropdown para seleccionar familia de producto
        html.Div([
            create_dropdown_with_label(
                label_text="Seleccione una familia",
                label_size="20px",
                dropdown_id="familia-selector",
                options=[{'label': i, 'value': i} for i in sorted(df['family'].unique())], # Las opciones serán los valores únicos de familias
                default_value=sorted(df['family'].unique())[0] # Por defecto, seleccionamos la primera en el dataframe
            ),
        ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'marginBottom': '10px'}),
        # Segunda línea: Selector de normalización (Absoluto/Relativo)
        html.Div([
            dcc.RadioItems(
                id='normalizacion-selector',
                options=[
                    {'label': 'Absoluto', 'value': 'absoluto'},
                    {'label': 'Relativo', 'value': 'relativo'}
                ],
                value='absoluto',  # Valor predeterminado
                inline=True,
                style={'display': 'flex', 'justifyContent': 'center', 'gap': '20px', 'fontSize': '16px'}
            )
        ], style={'textAlign': 'center', 'marginBottom': '20px'}),
        # Tercera línea: Comparación entre períodos y selección de años
        html.Div([
            # Comparación entre períodos
            html.Div([
                dcc.Checklist(
                    id='comparacion-selector',
                    options=[{'label': 'Comparar entre años', 'value': 'comparar'}],
                    value=[],
                    inline=True,
                    style={'fontSize': '16px', 'marginBottom': '10px'}
                )
            ], style={'marginRight': '20px'}),
            # Dropdowns para seleccionar los años (visibles solo si se selecciona comparar)
            html.Div([
                # Dropdown para seleccionar año 1
                create_dropdown_with_label(
                    label_text="Año 1",
                    label_size="16px",
                    dropdown_id="año-1-selector",
                    options=[{'label': str(año), 'value': año} for año in sorted(df['year'].unique())], # Presentamos como opciones los distintos años presentes en el daataset 
                    default_value=None
                ),
                # Dropdown para seleccionar año 2
                create_dropdown_with_label(
                    label_text="Año 2",
                    label_size="16px",
                    dropdown_id="año-2-selector",
                    options=[{'label': str(año), 'value': año} for año in sorted(df['year'].unique())],
                    default_value=None
                ),
            ], id='años-comparacion', style={'display': 'none', 'gap': '10px'})
        ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'marginBottom': '20px', 'gap': '20px'}),
        # Heatmap
        dcc.Graph(id='heatmap-patrones', style={'width': '100%', 'height': '450px'})
    ], style={
        'border': '1px solid #006d5b',
        'padding': '20px',
        'borderRadius': '10px',
        'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
        'width': '95%',
        'margin': 'auto',
        'marginBottom': '20px'
    }),

    # Comparación de estados con Radar Chart
    html.Div([
        html.H3("Comparación de rendimiento entre estados", style={'text-align': 'center', 'color': '#4b565e', 'fontSize': '30px'}),
        html.Div([
            html.Div([
                html.H3("Seleccione los estados a comparar:", style={'color': '#006d5b', 'fontSize': '20px', 'marginRight': '20px'}),
                # Dropdown para seleccionar varios estados a la vez (multi)
                dcc.Dropdown(
                    id="radar-estados-selector",
                    options=[{'label': state, 'value': state} for state in sorted(df['state'].unique())], # Presentaremos los distintos estados del df como opciones a seleccionar
                    value=["Pichincha", "Guayas"],  # Valores predeterminados
                    multi=True,  # Permitir múltiples selecciones
                    style={'width': '300px'}
                )
            ], style={
                'display': 'flex',
                'alignItems': 'center', 
                'justifyContent': 'center', 
                'marginBottom': '20px'
            }),
        ], style={'textAlign': 'center'}),
        dcc.Graph(
            id='grafico-radar-estados',
            style={'width': '100%', 'height': '500px'}
        )
    ], style={
        'border': '1px solid #006d5b',
        'padding': '20px',
        'borderRadius': '10px',
        'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
        'width': '95%',
        'margin': 'auto',
        'marginBottom': '20px'
    }),


        # Firma a pie de pagina
        firma_pag(),
        
    ], style={'background-color': '#eaf4eb', 'padding': '20px','border-radius': '20px', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)','margin-top': '10px'})

# Callbacks para actualizar los gráficos y la tabla al seleccionar una tienda del dropdown
@app.callback(
    Output('grafico-estacionalidad', 'figure'),
    [Input('dropdown', 'value'),
     Input('estacionalidad-variables', 'value'),
     Input('range-slider-estacionalidad', 'value')]
)
def update_estacionalidad(estado, variables, slider_range):
    # Convertimos el rango del slider en fechas específicas
    fecha_inicio = pd.to_datetime(slider_range[0], unit='s')
    fecha_fin = pd.to_datetime(slider_range[1], unit='s')

    # Filtramos los datos para el estado seleccionado dentro del rango de fechas
    df_filtrado = df.query("state == @estado and @fecha_inicio <= date <= @fecha_fin")

    # Extraemos las columnas numéricas válidas seleccionadas para el análisis
    columnas_validas = [col for col in variables if col in df_filtrado.select_dtypes(include=['number']).columns]
    df_resumen = df_filtrado.groupby('date', as_index=False)[columnas_validas].sum()

    # Creamos el gráfico de líneas para mostrar la estacionalidad
    fig = px.line(
        df_resumen, 
        x='date', 
        y=columnas_validas,
        labels={'value': 'Valor', 'date': 'Fecha'},
        title=f'Estacionalidad en el Estado de {estado}'
    )

    # Incorporamos días festivos como puntos adicionales si se seleccionan
    if 'holidays' in variables:
        festivos = (
            df_filtrado.query("holiday_type == 'Holiday'")
            .groupby('date', as_index=False)['sales']
            .sum()
        )

        # Añadimos marcadores para los festivos si existen datos
        if not festivos.empty:
            fig.add_scatter(
                x=festivos['date'],
                y=festivos['sales'],
                mode='markers',
                marker=dict(size=8, color='red', symbol='circle'),
                name='Festivos'
            )

    # Configuramos el diseño del gráfico para mantener coherencia visual
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='#4b565e'
    )

    return fig

@app.callback(
    Output('grafico-comparativo', 'figure'),
    [Input('dropdown', 'value'),
     Input('range-slider-comp', 'value'),
     Input('agrupacion-selector', 'value')]  # Selector para la agrupación
)
def update_comparativo(estado, slider_range, agrupacion):
    # Convertimos el rango del slider a fechas específicas
    fecha_inicio = pd.to_datetime(slider_range[0], unit='s')
    fecha_fin = pd.to_datetime(slider_range[1], unit='s')
    
    # Filtramos los datos por estado y rango de fechas
    df_filtrado = df.query("state == @estado and @fecha_inicio <= date <= @fecha_fin").copy()
    
    # Aseguramos que las columnas numéricas no contengan NaN y convertimos valores no válidos
    df_filtrado['transactions'] = pd.to_numeric(df_filtrado['transactions'], errors='coerce').fillna(0)
    df_filtrado['sales'] = pd.to_numeric(df_filtrado['sales'], errors='coerce').fillna(0)
    df_filtrado['onpromotion'] = pd.to_numeric(df_filtrado['onpromotion'], errors='coerce').fillna(0)
    
    # Verificamos si los datos filtrados están vacíos
    if df_filtrado.empty:
        return px.scatter().update_layout(
            title="No hay datos disponibles",
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)'
        )
    
    # Lógica para gráficos sin agrupación
    if agrupacion == 'none':
        # Agrupamos los datos a nivel de tienda
        df_agrupado = df_filtrado.groupby('store_nbr').agg(
            sales_avg=('sales', 'mean'),
            onpromotion_avg=('onpromotion', 'mean'),
            transactions_sum=('transactions', 'sum'),
            store_type=('store_type', 'first')  # Tipo de tienda asociado
        ).reset_index()
        
        # Creamos un gráfico de dispersión comparativo para las tiendas
        fig = px.scatter(
            df_agrupado,
            x='sales_avg',
            y='onpromotion_avg',
            size='transactions_sum',
            color='store_type',
            labels={
                'sales_avg': 'Ventas Promedio por Tienda',
                'onpromotion_avg': '% Promoción Promedio',
                'transactions_sum': 'Total de Transacciones',
                'store_type': 'Tipo de Tienda'
            },
            title=f'Comparación Multidimensional en el Estado de {estado}',
            hover_data=['store_nbr']
        )
    else:
        # Agrupamos los datos por clúster o tipo de tienda
        agrupacion_columna = 'store_type' if agrupacion == 'store_type' else 'cluster'
        
        # Verificamos que la columna de agrupación exista
        if agrupacion_columna not in df_filtrado.columns:
            return px.scatter().update_layout(
                title="No hay datos disponibles para la agrupación seleccionada",
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)'
            )
        
        # Calculamos las métricas agregadas según la agrupación
        df_agrupado = df_filtrado.groupby(agrupacion_columna).agg(
            sales_avg=('sales', 'mean'),
            onpromotion_avg=('onpromotion', 'mean'),
            transactions_sum=('transactions', 'sum')
        ).reset_index()
        
        # Verificamos si los datos agrupados están vacíos
        if df_agrupado.empty:
            return px.scatter().update_layout(
                title="No hay datos disponibles",
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)'
            )
        
        # Creamos un gráfico comparativo para la agrupación seleccionada
        fig = px.scatter(
            df_agrupado,
            x='sales_avg',
            y='onpromotion_avg',
            size='transactions_sum',
            color=agrupacion_columna,
            labels={
                'sales_avg': 'Ventas Promedio',
                'onpromotion_avg': '% Promoción Promedio',
                'transactions_sum': 'Total de Transacciones',
                agrupacion_columna: 'Agrupación'
            },
            title=f'Comparación Multidimensional en el Estado de {estado}',
            hover_data=['transactions_sum']
        )
    
    # Ajustamos la opacidad y configuramos el diseño visual del gráfico
    fig.update_traces(marker=dict(opacity=0.7))
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='#4b565e'
    )
    
    return fig

@app.callback(
    Output('heatmap-patrones', 'figure'),
    [Input('familia-selector', 'value'),  # Filtro por familia de producto
     Input('normalizacion-selector', 'value'),  # Selector de normalización
     Input('comparacion-selector', 'value'),  # Comparación entre períodos
     Input('año-1-selector', 'value'),  # Primer año para comparación
     Input('año-2-selector', 'value')]  # Segundo año para comparación
)
def update_heatmap(familia, normalizacion, comparacion, año_1, año_2):
    # Filtramos los datos según la familia seleccionada
    df_filtrado = df[df['family'] == familia].copy()

    # Verificamos si el DataFrame está vacío, devolviendo un heatmap vacío si no hay datos
    if df_filtrado.empty:
        empty_data = pd.DataFrame(
            data=[[0]], index=["Sin datos"], columns=["Sin datos"]
        )
        return px.imshow(
            empty_data,
            labels={'x': '', 'y': '', 'color': 'Sin datos'},
            title="No hay datos disponibles"
        ).update_layout(
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font_color='#4b565e'
        )

    # Agregamos columnas auxiliares para identificar día de la semana, semana y año
    df_filtrado['day_of_week'] = df_filtrado['date'].dt.day_name()
    df_filtrado['week'] = df_filtrado['date'].dt.isocalendar().week
    df_filtrado['year'] = df_filtrado['date'].dt.year

    # Implementamos lógica de comparación si la opción está habilitada y se especificaron años
    if 'comparar' in comparacion and año_1 and año_2:
        if año_1 == año_2:
            # Si los años son iguales, simplemente mostramos un heatmap para ese año
            df_año_1 = df_filtrado[df_filtrado['year'] == año_1]
            heatmap_data = df_año_1.groupby(['day_of_week', 'week'])['sales'].sum().reset_index()
            heatmap_data = heatmap_data.pivot(index='day_of_week', columns='week', values='sales').fillna(0)
            color_label = 'Ventas (%)' if normalizacion == 'relativo' else 'Ventas'
        else:
            # Filtramos los datos por los años seleccionados
            df_año_1 = df_filtrado[df_filtrado['year'] == año_1]
            df_año_2 = df_filtrado[df_filtrado['year'] == año_2]

            # Agrupamos las ventas por día y semana para ambos años
            heatmap_año_1 = df_año_1.groupby(['day_of_week', 'week'])['sales'].sum().reset_index()
            heatmap_año_2 = df_año_2.groupby(['day_of_week', 'week'])['sales'].sum().reset_index()

            # Normalizamos los datos si se seleccionó la opción de relativo
            if normalizacion == 'relativo':
                total_año_1 = heatmap_año_1['sales'].sum()
                total_año_2 = heatmap_año_2['sales'].sum()

                if total_año_1 > 0:
                    heatmap_año_1['sales'] = (heatmap_año_1['sales'] / total_año_1) * 100
                if total_año_2 > 0:
                    heatmap_año_2['sales'] = (heatmap_año_2['sales'] / total_año_2) * 100

            # Combinamos los datos de ambos años para generar la comparación
            heatmap_comparacion = pd.merge(
                heatmap_año_1, heatmap_año_2,
                on=['day_of_week', 'week'],
                how='outer',
                suffixes=(f'_{año_1}', f'_{año_2}')
            ).fillna(0)

            # Calculamos las diferencias entre ambos años (absolutas o porcentuales)
            heatmap_comparacion['diferencia'] = heatmap_comparacion[f'sales_{año_1}'] - heatmap_comparacion[f'sales_{año_2}']

            # Creamos la matriz para el heatmap basada en las diferencias calculadas
            heatmap_data = heatmap_comparacion.pivot(index='day_of_week', columns='week', values='diferencia').fillna(0)
            color_label = f'Diferencia de Ventas (%) ({año_1} vs {año_2})' if normalizacion == 'relativo' else f'Diferencia de Ventas ({año_1} vs {año_2})'
    else:
        # Normalizamos las ventas si se seleccionó la opción de relativo (sin comparación)
        if normalizacion == 'relativo':
            total_sales = df_filtrado['sales'].sum()
            if total_sales > 0:
                df_filtrado['sales'] = (df_filtrado['sales'] / total_sales) * 100

        # Creamos la matriz para el heatmap basada en datos normales o normalizados
        heatmap_data = df_filtrado.groupby(['day_of_week', 'week'])['sales'].sum().reset_index()
        heatmap_data = heatmap_data.pivot(index='day_of_week', columns='week', values='sales').fillna(0)
        color_label = 'Ventas (%)' if normalizacion == 'relativo' else 'Ventas'

    # Creamos el gráfico de heatmap con los datos procesados
    fig = px.imshow(
        heatmap_data,
        labels={'x': 'Semana del Año', 'y': 'Día de la Semana', 'color': color_label},
        title=f'Patrones de Venta ({familia})'
    )

    # Configuramos la estética del gráfico para alinearlo con el diseño general
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font_color='#4b565e'
    )

    return fig

@app.callback(
    Output('años-comparacion', 'style'),  # Modificamos el estilo del contenedor de selección de años
    Input('comparacion-selector', 'value')  # Detectamos el valor seleccionado en el comparador
)
def toggle_año_dropdowns(comparacion_value):
    # Verificamos si la opción de comparación está activa
    if 'comparar' in comparacion_value:
        # Mostramos el selector de años con un diseño flexible y espacio entre elementos
        return {'display': 'flex', 'gap': '10px'}
    # Ocultamos el selector de años si no se selecciona la comparación
    return {'display': 'none'}

@app.callback(
    Output('grafico-radar-estados', 'figure'),  # Salida: figura del gráfico radar
    Input('radar-estados-selector', 'value')  # Entrada: lista de estados seleccionados
)
def update_radar_estados(estados):
    # Verificamos si hay estados seleccionados
    if not estados or len(estados) == 0:
        # Devolvemos un gráfico vacío con un mensaje si no se seleccionan estados
        return px.scatter().update_layout(
            title="No hay estados seleccionados",
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font_color='#4b565e'
        )

    # Filtramos los datos para incluir únicamente los estados seleccionados
    df_filtrado = df[df['state'].isin(estados)].copy()

    # Verificamos si los datos filtrados están vacíos
    if df_filtrado.empty:
        # Devolvemos un gráfico vacío con un mensaje si no hay datos disponibles
        return px.scatter().update_layout(
            title="No hay datos disponibles para los estados seleccionados",
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font_color='#4b565e'
        )

    # Calculamos métricas agregadas por estado
    resumen_estados = df_filtrado.groupby('state').agg(
        Ventas=('sales', 'sum'),  # Total de ventas por estado
        Promociones_Activas=('onpromotion', 'sum'),  # Total de promociones activas
        Transacciones=('transactions', 'sum')  # Total de transacciones
    ).reset_index()

    # Calculamos métricas adicionales para el análisis
    resumen_estados['Ventas_Promedio_Transaccion'] = (
        resumen_estados['Ventas'] / resumen_estados['Transacciones']
    ).fillna(0)  # Calculamos el promedio de ventas por transacción y manejamos valores nulos

    resumen_estados['Promociones_Por_Transaccion'] = (
        resumen_estados['Promociones_Activas'] / resumen_estados['Transacciones']
    ).fillna(0)  # Calculamos el promedio de promociones por transacción y manejamos valores nulos

    # Normalizamos las métricas en una escala de 0 a 100
    for col in ['Ventas', 'Promociones_Activas', 'Transacciones', 'Ventas_Promedio_Transaccion', 'Promociones_Por_Transaccion']:
        max_val = resumen_estados[col].max()  # Encontramos el valor máximo en la métrica
        if max_val > 0:
            resumen_estados[col] = (resumen_estados[col] / max_val) * 100  # Normalizamos el valor

    # Reestructuramos los datos para que se adapten al formato del gráfico radar
    radar_data = resumen_estados.melt(
        id_vars='state',  # La variable de identificación es el estado
        var_name='Métrica',  # Nombramos la columna de métricas
        value_name='Valor'  # Nombramos la columna de valores
    )

    # Creamos el gráfico de radar para comparar el rendimiento por estado
    fig = px.line_polar(
        radar_data,
        r='Valor',  # Valor radial (métrica normalizada)
        theta='Métrica',  # Las métricas en el eje angular
        color='state',  # Coloreamos por estado
        line_close=True,  # Cerramos las líneas del radar
        title="Comparación de rendimiento por estado"  # Título del gráfico
    )

    # Configuramos el diseño del gráfico para mantener coherencia estética
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 100])  # Definimos la escala del eje radial
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Fondo transparente
        paper_bgcolor='rgba(0,0,0,0)',  # Fondo transparente del gráfico
        font_color='#4b565e'  # Color del texto
    )

    # Devolvemos el gráfico generado
    return fig


## Pestaña Explicativa

In [180]:
# Pestaña explicativa
tab_4_content = html.Div([
    # Título principal
    title("Documentación del Dashboard"),
    
    # Línea divisoria
    html.Br(),
    
    # Explicación de la pestaña de visualización básica
    create_section("Resumen de Métricas", [
        "Esta primera pestaña del dashboard está diseñada para proporcionar una visión general rápida y clara del rendimiento de la empresa. Se enfoca en exponer "
        "tanto métricas clave como tendencias generales, distribuidas en gráficos que cubren diferentes dimensiones del análisis. Para ello hace uso de:",
        html.Ul([
            html.Li([
                html.Span("Contador de métricas básicas: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Esta sección calcula tres datos clave: el número total de tiendas, productos y estados en los que la empresa opera. Estos valores se obtienen "
                "a partir de registros únicos en el dataset, lo que ofrece una visión global de las capacidades de la empresa. Los resultados son mostrados de forma clara y directa."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("10 Productos más vendidos: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Un gráfico de barras que muestra los diez productos con mayor volumen de ventas. Para crearlo, se calculan las ventas promedio por familia de producto "
                "y se seleccionan las diez con mayores valores, excluyendo datos faltantes o inconsistentes. Esto asegura un análisis preciso y enfocado."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Ventas mensuales: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Un gráfico de líneas que visualiza la evolución de las ventas mes a mes. Las ventas totales por mes se agrupan y convierten a millones para simplificar la interpretación. "
                "Se utilizan líneas y marcadores para resaltar los picos estacionales y facilitar la identificación de tendencias."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Ventas por día de la semana: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este gráfico detalla las ventas promedio según el día de la semana, permitiendo identificar los días con mayor actividad. "
                "Los datos se agrupan por día y se representan en un gráfico de líneas con marcadores para resaltar los patrones."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Ventas por estado: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Un gráfico de barras que desglosa las ventas por estado, destacando las regiones con mayor rendimiento. Los datos se ordenan en orden descendente "
                "para facilitar el análisis visual de los estados más relevantes."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Ventas en días laborales y festivos: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Un gráfico comparativo que analiza las diferencias de ventas entre días laborales y festivos. Las ventas se agrupan por tipo de día y se representan "
                "en un gráfico de barras para visualizar cómo el contexto afecta el comportamiento de los consumidores."
            ], style={'marginBottom': '10px'}),
        ])
    ]),
   
    # Línea divisoria
    html.Br(),
    
    # Explicación de la pestaña de interactividad simple
    create_section("Análisis por Tienda", [
        "La segunda pestaña se centra en el rendimiento específico de cada tienda de la empresa. Esta sección es especialmente útil para "
        "analizar cómo las operaciones individuales contribuyen al rendimiento general y detectar patrones relevantes que pueden guiar decisiones locales.",
        "Incluye los siguientes componentes principales:",
        html.Ul([
            html.Li([
                html.Span("Menú desplegable: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este menú permite seleccionar una tienda específica. Los identificadores únicos de las tiendas se extraen del dataset y "
                "se limpian para eliminar duplicados o inconsistencias. El diseño asegura accesibilidad, y el menú interactúa con los gráficos "
                "para mostrar datos relacionados con la tienda seleccionada."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Gráfico de ventas anuales por tienda: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este gráfico de barras se crea agrupando las ventas totales por año para la tienda seleccionada. Las ventas se convierten a millones "
                "para facilitar la interpretación. El gráfico utiliza una paleta de colores personalizada y es interactivo, permitiendo zoom, restablecimiento "
                "de vistas y la opción de descarga como imagen. Los datos de años incompletos se manejan para asegurar consistencia en la representación visual."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Gráfico de ventas por producto: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Desglosa las ventas por familia de productos mediante un gráfico de barras horizontal. Los datos se agrupan por familia y se transforman a millones "
                "para mantener coherencia visual. Los colores se utilizan para diferenciar las familias de productos, y el diseño permite identificar fácilmente "
                "las familias más importantes en términos de ventas."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Gráfico de productos en promoción: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este gráfico de barras horizontal muestra las ventas de productos en promoción, agrupadas por familia. Los datos se filtran para incluir solo productos "
                "en promoción, y las ventas se convierten a millones. La visualización ayuda a analizar cómo las promociones impactan las ventas y a identificar "
                "familias de productos clave para futuras estrategias promocionales."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Tabla resumen de productos en promoción: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "La tabla presenta un resumen detallado de las familias de productos promocionados. Se muestran el porcentaje de ventas en promoción de cada familia "
                "y el número total de promociones activas. Los datos se normalizan para destacar las contribuciones relativas de cada familia. Además, la tabla incluye "
                "funcionalidades de paginación y colores para mejorar la legibilidad."
            ], style={'marginBottom': '10px'})
        ])
    ]),

    # Línea divisoria
    html.Br(),
    
    # Explicación de la pestaña de análisis avanzado
    create_section("Análisis Avanzado", [
        "La tercera pestaña está diseñada para ofrecer un análisis exhaustivo que combina patrones estacionales, comparaciones multidimensionales y análisis de patrones de venta. Cada gráfico incluye características interactivas y opciones de personalización que permiten un entendimiento profundo de las dinámicas de ventas y promociones.",
        "La pestaña incluye las siguientes funcionalidades clave:",
        html.Ul([
            html.Li([
                html.Span("Análisis de Estacionalidad: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este gráfico interactivo permite observar cómo las ventas diarias, los productos en promoción y los días festivos varían a lo largo del tiempo para un estado seleccionado.",
                html.Ul([
                    html.Li([
                        html.Span("Filtros dinámicos: ", style={'fontWeight': 'bold'}),
                        "Los datos se filtran por estado usando un menú desplegable y se selecciona un rango temporal a través de un slider interactivo."
                    ]),
                    html.Li([
                        html.Span("Variables visualizables: ", style={'fontWeight': 'bold'}),
                        "Un checklist permite al usuario elegir entre ventas diarias, productos en promoción y días festivos para incluir en el gráfico."
                    ]),
                    html.Li([
                        html.Span("Formato del gráfico: ", style={'fontWeight': 'bold'}),
                        "Se utiliza un gráfico de líneas para las ventas y promociones, con los días festivos representados como puntos rojos marcados en la línea de tiempo."
                    ]),
                    html.Li([
                        html.Span("Interactividad: ", style={'fontWeight': 'bold'}),
                        "Los usuarios pueden acercar o alejar áreas específicas para un análisis detallado, restablecer vistas, y descargar el gráfico como imagen."
                    ]),
                    html.Li([
                        html.Span("Estilo visual: ", style={'fontWeight': 'bold'}),
                        "El gráfico incluye una paleta de colores coherente con la identidad visual del dashboard, y los puntos de los días festivos destacan con un color rojo intenso."
                    ])
                ])
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Análisis Comparativo Multidimensional: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este gráfico de burbujas está diseñado para comparar métricas clave como las ventas promedio, el porcentaje de productos en promoción y el total de transacciones.",
                html.Ul([
                    html.Li([
                        html.Span("Opciones de agrupación: ", style={'fontWeight': 'bold'}),
                        "Un menú desplegable permite agrupar los datos por tipo de tienda o por clúster, ajustando los colores de las burbujas en consecuencia."
                    ]),
                    html.Li([
                        html.Span("Ejes principales: ", style={'fontWeight': 'bold'}),
                        "El eje X representa las ventas promedio, mientras que el eje Y muestra el porcentaje de productos en promoción. El tamaño de las burbujas indica el volumen total de transacciones."
                    ]),
                    html.Li([
                        html.Span("Interactividad: ", style={'fontWeight': 'bold'}),
                        "Incluye funcionalidades de zoom, selección de puntos (box y lasso), y la capacidad de descargar la visualización."
                    ]),
                    html.Li([
                        html.Span("Leyenda: ", style={'fontWeight': 'bold'}),
                        "Proporciona información sobre los colores según la agrupación seleccionada, permitiendo destacar u ocultar categorías específicas con un solo clic."
                    ]),
                    html.Li([
                        html.Span("Estilo visual: ", style={'fontWeight': 'bold'}),
                        "Se utiliza una cuadrícula ligera para mantener el enfoque en los datos, con colores optimizados para una clara diferenciación."
                    ])
                ])
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Patrones de Venta: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este mapa de calor visualiza las ventas según el día de la semana y la semana del año, destacando patrones temporales significativos.",
                html.Ul([
                    html.Li([
                        html.Span("Ejes del gráfico: ", style={'fontWeight': 'bold'}),
                        "El eje Y representa los días de la semana, mientras que el eje X muestra las semanas del año."
                    ]),
                    html.Li([
                        html.Span("Colores: ", style={'fontWeight': 'bold'}),
                        "La intensidad del color refleja los volúmenes de ventas. Los usuarios pueden optar por normalizar los datos a valores absolutos o relativos."
                    ]),
                    html.Li([
                        html.Span("Opciones de filtrado: ", style={'fontWeight': 'bold'}),
                        "Permite seleccionar un año base y una familia de productos específica mediante menús desplegables."
                    ]),
                    html.Li([
                        html.Span("Comparación entre años: ", style={'fontWeight': 'bold'}),
                        "Si se selecciona la opción de comparar años y se eligen dos años distintos, se muestra un mapa de calor comparativo en el que los colores indican las diferencias entre ambos años (aumentos en azul, disminuciones en rojo)."
                    ]),
                    html.Li([
                        html.Span("Caso especial: ", style={'fontWeight': 'bold'}),
                        "Si la comparación está habilitada pero el mismo año es seleccionado para ambos campos, el gráfico mostrará únicamente el mapa de calor para ese año, sin realizar comparación."
                    ]),
                    html.Li([
                        html.Span("Visualización comparativa: ", style={'fontWeight': 'bold'}),
                        "Los colores se ajustan para mostrar aumentos (azul) o disminuciones (rojo) en las ventas al comparar dos años."
                    ]),
                    html.Li([
                        html.Span("Interactividad: ", style={'fontWeight': 'bold'}),
                        "La visualización es completamente interactiva, permitiendo explorar las celdas en detalle y descargar el gráfico como imagen."
                    ])
                ])
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Comparación de Rendimiento entre Estados: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Este gráfico radar permite comparar métricas clave como ventas totales, promociones activas, transacciones realizadas y promedio de ventas por transacción entre varios estados.",
                html.Ul([
                    html.Li([
                        html.Span("Selección de estados: ", style={'fontWeight': 'bold'}),
                        "Un menú desplegable permite al usuario elegir múltiples estados a analizar."
                    ]),
                    html.Li([
                        html.Span("Normalización de datos: ", style={'fontWeight': 'bold'}),
                        "Todas las métricas se escalan de 0 a 100 para facilitar la comparación."
                    ]),
                    html.Li([
                        html.Span("Datos representados: ", style={'fontWeight': 'bold'}),
                        "Cada métrica se visualiza en un eje del radar, proporcionando una visión integral de las fortalezas y debilidades de cada estado."
                    ]),
                    html.Li([
                        html.Span("Estilo visual: ", style={'fontWeight': 'bold'}),
                        "Se utilizan colores consistentes con la identidad del dashboard y un fondo transparente para destacar los datos."
                    ])
                ])
            ], style={'marginBottom': '10px'})
        ])
    ]),

    # Línea divisoria
    html.Br(),
    
    # Explicación de decisiones de diseño
    create_section("Decisiones de diseño", [
        "El diseño del dashboard sigue principios claros de simplicidad, funcionalidad y estética:",
        html.Ul([
            html.Li([
                html.Span("Simplicidad: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Cada pestaña está estructurada de manera lógica, permitiendo a los usuarios centrarse en las métricas y gráficos relevantes sin distracciones."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Funcionalidad: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Se priorizó la interactividad en todas las visualizaciones, incluyendo opciones como filtros, zoom y tooltips. Esto permite un análisis "
                "dinámico y adaptado a las necesidades específicas de cada usuario."
            ], style={'marginBottom': '10px'}),
            html.Li([
                html.Span("Estética: ", style={'color': '#26a69a','fontWeight': 'bold'}),
                "Los colores y estilos utilizados se seleccionaron para garantizar una experiencia visual agradable y profesional. Los bordes redondeados y "
                "las sombras sutiles aportan una apariencia moderna y pulida."
            ], style={'marginBottom': '10px'})
        ]),
        "\nFinalmente, el código está completamente documentado para facilitar su comprensión y futura ampliación. Cada cálculo o funcionalidad implementada "
        "ha sido explicada con detalle, asegurando transparencia en todo el desarrollo."
    ]),
    
    # Línea divisoria
    html.Br(),
    
    # Firma a pie de página
    firma_pag()

], style={'background-color': '#eaf4eb', 'padding': '20px','border-radius': '20px', 'box-shadow': '0px 4px 8px rgba(0, 0, 0, 0.1)','margin-top': '10px'})



# 6. Estructura

Una vez definido el contenido de cada pestaña unificamos el código

In [181]:
@app.callback(Output('tabs-content-example-graph', 'children'),Input('tabs-example-graph', 'value'))

def render_content(tab):
    if tab == 'tab_1':
        return tab_1_content
    elif tab == 'tab_2':
        return tab_2_content
    elif tab== 'tab_3':
        return tab_3_content
    elif tab== 'tab_4':
        return tab_4_content

# 7. Ejecución

Por último, iniciamos el servidor de la aplicación.

In [182]:
if __name__ == '__main__':
    app.run_server(debug=True,jupyter_mode = 'external', port=5001)

Dash app running on http://127.0.0.1:5001/
